In [1]:
import numpy as np

import illustris_python as il
import data as ilData #need to put path to data.py if in other file

import requests
import h5py
import bokeh.plotting as blt
import bokeh
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
sim = ilData.getSim('Illustris-1')
whichSnap=135
snap = ilData.getSnap(sim,-1+(whichSnap-135))

KeyboardInterrupt: 

In [8]:
minMass=10^11
maxMass=10^12
search_query="?mass__gt="+str(minMass)+"&mass__lt="+str(maxMass)
subs=ilData.get(snap['subhalos']+search_query)

sub=ilData.getSub(subs,0)
print(sub)

cut_req={'dm':'Coordinates,Velocities'}
cutout=ilData.get(sub[''['url']+'cutout.hdf5',cut_req)
with h5py.File(cutout,'r') as f:
    x = f['PartType1']['Coordinates'][:,0] - sub['pos_x'] # ckpc/h
    y = f['PartType1']['Coordinates'][:,1] - sub['pos_y']
    z = f['PartType1']['Coordinates'][:,2] - sub['pos_z']
    vx = f['PartType1']['Velocities'][:,0] - sub['vel_x'] # km/s
    vy = f['PartType1']['Velocities'][:,1] - sub['vel_y']
    vz = f['PartType1']['Velocities'][:,2] - sub['vel_z']

{'halfmassrad_stars': 4.7584, 'parent': 0, 'stellarphotometricsrad': 6.4313, 'prog_sfid': 75, 'gasmetallicitysfr': 0.0, 'gasmetallicitysfrweighted': 0.0, 'starmetallicity': 0.0290835, 'starmetallicityhalfrad': 0.0321282, 'stellarphotometricsmassinrad': 1.97596, 'spin_y': 70.217, 'primary_flag': 0, 'halfmassrad_gas': 0.0, 'vmax': 137.855, 'massinmaxrad_dm': 1.23231, 'stellarphotometrics_r': -20.643, 'massinmaxrad': 3.12193, 'bhmdot': 0.0, 'massinhalfrad_dm': 0.643709, 'halfmassrad_dm': 6.9033, 'mass_dm': 2.37996, 'stellarphotometrics_b': -19.3433, 'vel_x': 97.8794, 'mass_gas': 0.0, 'massinmaxrad_bhs': 0.0, 'desc_snap': -1, 'massinmaxrad_gas': 0.0, 'windmass': 0.0, 'trees': {'lhalotree_mpb': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/74/lhalotree/mpb.hdf5', 'lhalotree': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/74/lhalotree/full.hdf5', 'sublink': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/74/sublin

KeyError: 'url'

In [13]:
pos=np.vstack((x,y,z))/0.7 #ckpc (no h!)
vel=np.vstack((vx,vy,vz))
rad=np.linalg.norm(pos,axis=0)
nDot=rad.size
vr=np.zeros(nDot)
vt=np.zeros(nDot)
for i in range(nDot):
    #print('r hat: ',pos[:,i]/rad[i])
    #print('vel: ',vel[:,i])
    vr[i]=np.dot(pos[:,i],vel[:,i].T)/rad[i]
    #print('vr: ',vr[i])
    vt[i]=np.linalg.norm(np.cross(pos[:,i],vel[:,i])/rad[i])
    #print('vt: ',vt[i])

In [14]:
maxR=20
minR=1
nR=10
bins=np.linspace(np.sqrt(minR),np.sqrt(maxR),nR)**2
avr=np.zeros(nR-1)
svr=np.zeros(nR-1)
avt=np.zeros(nR-1)
svt=np.zeros(nR-1)
for i in range(nR-1):
    whichRad,=np.where((rad>bins[i]) & (rad<bins[i+1]))
    nWhich=whichRad.size
    if nWhich==0:
        continue
    avr[i]=np.mean(vr[whichRad])
    avt[i]=np.mean(vt[whichRad])
    svr[i]=np.std(vr[whichRad])#/np.sqrt(nWhich)
    svt[i]=np.std(vt[whichRad])#/np.sqrt(nWhich)
    

In [15]:
def errorPatch(x,y,s):
    pS=y+s
    mS=y-s
    #print('av: ',y)
    #print('max: ',pS)
    #print('min: ',mS)
    return np.hstack((x,x[::-1])),np.hstack((pS,mS[::-1]))
plot=blt.figure()
rBin=(bins[1:]+bins[:-1])/2
rX,vtY=errorPatch(rBin,avt,svt)
rX,vrY=errorPatch(rBin,avr,svr)

plot.line(rBin,avr,color='blue')
plot.line(rBin,avt,color='red')
plot.patch(rX,vtY,alpha=0.2,color='red')
plot.patch(rX,vrY,alpha=0.2,color='blue')
plot.xaxis.axis_label='radius'
plot.yaxis.axis_label='velocity'

blt.show(plot)

In [145]:
plot2=blt.figure()
plot2.circle(x,y)
blt.show(plot2)

In [144]:
print(sub['len_dm'])

5940
